In [1]:
import keras.metrics
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from tensorflow.keras import models, layers
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, LSTM, Reshape, BatchNormalization, LeakyReLU
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
import keras
import tensorflow as tf
import os
from sklearn.utils import class_weight

/Users/helenamehler/PycharmProjects/fallDetection/venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
def windowing(x, y, window_size, overlap, fall = 2):
    windows = []
    y_labels = []

    step_size = window_size - overlap

    for i in range(0, len(x) - window_size + 1, step_size):
        window = x.iloc[i:(i + window_size), :]
        windows.append(window)
        if np.sum(y[i:i + window_size] >= fall) >= 30:
            y_labels.append(1)
        else:
            y_labels.append(0)

    return np.array(windows), np.array(y_labels)

In [3]:
data_path = '../../dataSets/SisFall_labelled'
dfs = []

for root, dirs, files in os.walk(data_path):
    for file in files:
        file_path = os.path.join(root, file)
        if '.csv' in file:
            dfs.append(pd.read_csv(file_path, index_col=0))

In [4]:
window_size = 200
overlap = 50

In [39]:
windowed_x = []
windowed_y = []
for i, dataset in enumerate(dfs):
    x_w, y_w = windowing(dataset.drop(columns = ['fall']), dataset['fall'], window_size, overlap)
    windowed_x.extend(x_w)
    windowed_y.extend(y_w)

In [40]:
x_train = []
y_train = []
x_test = []
y_test = []
for i in range(len(windowed_x)):
    if np.random.rand(1)[0] < 0.2:
        x_test.append(windowed_x[i])
        y_test.append(windowed_y[i])
    else:
        x_train.append(windowed_x[i])

        y_train.append(windowed_y[i])

In [49]:
x_train_modified = [(arr * (2 * 16) / (2 ** 13)) for arr in x_train]
x_train_cut = [np.clip(arr, -4, 4) for arr in x_train_modified]


x_test_modified = [(arr * (2 * 16) / (2 ** 13)) for arr in x_test]
x_test_cut = [np.clip(arr, -4, 4) for arr in x_test_modified]

In [7]:
scaler = MinMaxScaler((-4, 4))

x_train = np.array(x_train)
x_train = scaler.fit_transform(x_train.reshape(-1, x_train.shape[-1])).reshape(x_train.shape)

x_test = np.array(x_test)
x_test = scaler.transform(x_test.reshape(-1, x_test.shape[-1])).reshape(x_test.shape)

y_train = np.array(y_train)
y_test = np.array(y_test)

In [54]:
y_trainCat = to_categorical(y_train, num_classes=2)
input_shape = (200, 3)

In [59]:
class_weights = class_weight.compute_class_weight('balanced', classes = np.unique(y_train),y = y_train)
weight = {i : class_weights[i] for i in range(len(class_weights))}

trying out different models, but best one is still the first one

In [50]:
model = models.Sequential()
model.add(layers.Conv1D(6, kernel_size=4, activation='relu', input_shape=input_shape, padding='same', name='Accelerometer'))

model.add(layers.BatchNormalization())

model.add(layers.MaxPooling1D(pool_size=4, strides=4))

model.add(layers.Conv1D(12, kernel_size=4, activation='relu', padding='same'))

model.add(layers.BatchNormalization())

model.add(layers.MaxPooling1D(pool_size=4, strides=4))
model.add(layers.Flatten())

model.add(layers.Dense(128, activation='relu'))
model.add(layers.BatchNormalization())

model.add(layers.Dropout(0.3))
model.add(layers.Dense(2, activation='softmax', name='Fall'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', keras.metrics.AUC(), keras.metrics.Precision(), keras.metrics.Recall()])


In [90]:
model = models.Sequential()
model.add(layers.Conv1D(6, kernel_size=4, activation='relu', input_shape=input, padding='same'))

model.add(layers.MaxPooling1D(pool_size=4, strides=4))

model.add(layers.Conv1D(12, kernel_size=4, activation='relu', padding='same'))

model.add(layers.MaxPooling1D(pool_size=4, strides=4))
model.add(layers.Flatten())

model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(2, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [45]:
model = models.Sequential()
model.add(Conv1D(filters=64, kernel_size=4, activation='relu', input_shape=(128,3)))
model.add(Conv1D(filters=64, kernel_size=4, activation='relu'))
model.add(Dropout(0.5))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(2, activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [84]:
model = models.Sequential()

# Convolutional layer
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(128, 3)))
model.add(MaxPooling1D(pool_size=2))

# LSTM layer
model.add(LSTM(20, activation='relu'))

# Dense layer for classification
model.add(Dense(2, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

#other metric: metrics=[tf.keras.metrics.FalseNegatives(),tf.keras.metrics.FalsePositives(),'accuracy'] 

In [55]:
x_train_cut = np.array(x_train_cut)
batch = 80
epochs = 20
iterationsPerEp = x_train_cut.shape[0] // batch
model.fit(x_train_cut, y_trainCat, batch_size=batch, epochs=epochs, steps_per_epoch=iterationsPerEp)

Epoch 1/20
1010/1010 [==============================] - 6s 5ms/step - loss: 0.1117 - accuracy: 0.9633 - auc_2: 0.9915 - precision_2: 0.9633 - recall_2: 0.9633
Epoch 2/20
1010/1010 [==============================] - 5s 5ms/step - loss: 0.0298 - accuracy: 0.9917 - auc_2: 0.9984 - precision_2: 0.9917 - recall_2: 0.9917
Epoch 3/20
1010/1010 [==============================] - 5s 5ms/step - loss: 0.0247 - accuracy: 0.9930 - auc_2: 0.9989 - precision_2: 0.9930 - recall_2: 0.9930
Epoch 4/20
1010/1010 [==============================] - 5s 5ms/step - loss: 0.0215 - accuracy: 0.9938 - auc_2: 0.9991 - precision_2: 0.9938 - recall_2: 0.9938
Epoch 5/20
1010/1010 [==============================] - 5s 5ms/step - loss: 0.0198 - accuracy: 0.9942 - auc_2: 0.9992 - precision_2: 0.9942 - recall_2: 0.9942
Epoch 6/20
1010/1010 [==============================] - 6s 6ms/step - loss: 0.0184 - accuracy: 0.9941 - auc_2: 0.9993 - precision_2: 0.9941 - recall_2: 0.9941
Epoch 7/20
1010/1010 [========================

In [57]:
x_test_cut = np.array(x_test_cut)
predictions = model.predict(x_test_cut)
yPredClasses = np.argmax(predictions, axis=1)
confMat = confusion_matrix(y_test, yPredClasses)
confMat

636/636 [==============================] - 1s 843us/step


array([[19737,    43],
       [   61,   487]])

In [58]:
print('Accuracy', accuracy_score(y_test, yPredClasses))
print('F1-Score', f1_score(y_test, yPredClasses))
print('AUC', roc_auc_score(y_test, yPredClasses))

Accuracy 0.9948839039748131
F1-Score 0.9035250463821893
AUC 0.9432561091716914


In [71]:
self_collected = pd.read_csv('../../dataSets/Self_collected/paul_fall_2.csv', header=None)
self_collected = self_collected.iloc[:, :3]
#zero_rows = pd.DataFrame(0, index=range(20), columns=self_collected.columns)
#self_collected = pd.concat([self_collected, zero_rows], ignore_index=True)

y = np.zeros(len(self_collected))
windowed_real_x, windowed_real_y = windowing(self_collected, y, window_size, overlap)

In [72]:
predictions_test = model.predict(windowed_real_x)
yPredClasses_test = np.argmax(predictions_test, axis=1)
confMat = confusion_matrix(windowed_real_y, yPredClasses_test)
confMat

1/1 [==============================] - 0s 12ms/step


array([1])